<a href="https://colab.research.google.com/github/Nastya880/cuda/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Лабораторная 3**

Гистограмма

Дан массив А из N натуральных элементов от 0 до 255 с нормальным распределением. Построить гистограмму, содержащую число каждого элемента массива.

In [1]:
%%writefile histogram.cu
#include <cassert>
#include <cstring>
#include <random>
#include <cstdio>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

typedef unsigned int uint;
typedef unsigned char uchar;
constexpr auto n = (1024 * 2); 
constexpr auto log2_warp_size = 5;
constexpr auto warp_size = 32; 
constexpr auto tag_mask = 0x07FFFFFFU; 
constexpr auto num_bins = 256; 
constexpr auto num_warps = 6; 
constexpr auto merge_threadblock_size = 256;

inline __device__ void add_byte(volatile uint* warp_hist, const uint data, uint thread_tag)
{
	uint count;
	do
	{// прочесть текущее значение счетчика и снять идентификатор нити 
		count = warp_hist[data] & tag_mask;
		count = thread_tag | (count + 1);// увеличить его на единицу и поставить свой идентификатор
		warp_hist[data] = count;//осуществить запись
	} while (warp_hist[data] != count);//пока запись не прошла успешно
}

inline __device__ void add_word(volatile uint* warp_hist, const uint data, const uint tag)
{
	add_byte(warp_hist, (data >> 0) & 0xFFU, tag);
	add_byte(warp_hist, (data >> 8) & 0xFFU, tag);
	add_byte(warp_hist, (data >> 16) & 0xFFU, tag);
	add_byte(warp_hist, (data >> 24) & 0xFFU, tag);
}

__global__ void histogram_kernel(uint* partial_histograms, const uint* data, const uint data_count)
{
	__shared__ uint hist[num_bins * num_warps];
	uint* warpHist = hist + (threadIdx.x >> log2_warp_size) * num_bins;

#pragma unroll
	for (uint i = 0; i < num_bins / warp_size; i++)
		hist[threadIdx.x + i * num_warps * warp_size/*число нитей в блоке=192*/] = 0;

	uint tag = threadIdx.x << (32 - log2_warp_size);//айди нити
	__syncthreads();


	for (uint pos = blockIdx.x * blockDim.x + threadIdx.x; pos < data_count;
		pos += blockDim.x * gridDim.x)
	{
		uint d = data[pos];
		add_word(warpHist, d, tag);
	}
	__syncthreads();
	// объединить гистограммы данного блока и записать результат в глобальную память
	// 192 нити суммируют данные до 256 элементов гистограмм

	for (uint bin = threadIdx.x; bin < num_bins; bin += num_warps * warp_size)
	{
		uint sum = 0;
		for (uint i = 0; i < num_warps; i++)
			sum += hist[bin + i * num_bins] & tag_mask;
		partial_histograms[blockIdx.x * num_bins + bin] = sum;
	}
}

__global__ void merge_histogram_kernel(uint* out_histogram, const uint* partial_histograms, const uint histogram_count)
{
	uint sum = 0;
	for (uint i = threadIdx.x; i < histogram_count; i += 256)
		sum += partial_histograms[blockIdx.x + i * num_bins];
	__shared__ uint data[num_bins];
	data[threadIdx.x] = sum;
	for (uint stride = num_bins / 2; stride > 0; stride >>= 1)
	{
		__syncthreads();
		if (threadIdx.x < stride)
			data[threadIdx.x] += data[threadIdx.x + stride];
	}
	if (threadIdx.x == 0)
		out_histogram[blockIdx.x] = data[0];
}

void histogram(uint* histogram, void* data_dev, const uint byteCount)
{
	assert(byteCount % 4 == 0);
	const int n = byteCount / 4;
	int numBlocks = n / (num_warps * warp_size);
	constexpr int numPartials = 240;
	uint* partialHistograms = nullptr;
	cudaMalloc((void**)&partialHistograms, numPartials * num_bins * sizeof(uint));
	histogram_kernel << <dim3(numPartials), dim3(num_warps * warp_size) >> > (
		partialHistograms, (uint*)data_dev, n);
	merge_histogram_kernel << <dim3(num_bins), dim3(256) >> > (histogram,
		partialHistograms, numPartials);
	cudaFree(partialHistograms);
}

void randomInit(uint* a, int n, uint* h)
{
	std::mt19937 gen(1607);
	std::normal_distribution<> distr(128, 32);

	for (int i = 0; i < n; i++)
	{
		const uchar b1 = static_cast<int>(distr(gen)) & 0xFF;
		const uchar b2 = static_cast<int>(distr(gen)) & 0xFF;
		const uchar b3 = static_cast<int>(distr(gen)) & 0xFF;
		const uchar b4 = static_cast<int>(distr(gen)) & 0xFF;
		a[i] = b1 | (b2 << 8) | (b3 << 16) | (b4 << 24);
		h[b1]++;
		h[b2]++;
		h[b3]++;
		h[b4]++;
	}
}
int main(int argc, char* argv[])
{
	const auto a = new uint[n];
	uint* h_dev = nullptr;
	uint* a_dev = nullptr;
	uint h[num_bins];
	uint h_host[num_bins];
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
	memset(h_host, 0, sizeof(h_host));
	randomInit(a, n, h_host);
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, nullptr);
	cudaMalloc(reinterpret_cast<void**>(&a_dev), n * sizeof(uint));
	cudaMalloc(reinterpret_cast<void**>(&h_dev), num_bins * sizeof(uint));
	cudaMemcpy(a_dev, a, n * sizeof(uint), cudaMemcpyHostToDevice);
	histogram(h_dev, a_dev, 4 * n);
	cudaMemcpy(h, h_dev, num_bins * sizeof(uint), cudaMemcpyDeviceToHost);
	cudaFree(a_dev);
	cudaFree(h_dev);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("Elapsed time: %.2f\n", gpu_time);
	for (int i = 0; i < num_bins; i++)
	{
		for (int j = 0; j < h[i]; j++)
		{
			printf("*");
		}
		printf("\n");
	}
	delete[] a;
	return 0;
}


Writing histogram.cu


In [2]:
!nvcc histogram.cu -o histogram -Wno-deprecated-gpu-targets
!nvprof ./histogram

histogram.cu(90): warning: variable "numBlocks" was declared but never referenced

histogram.cu(16): warning: variable "merge_threadblock_size" was declared but never referenced

==567== NVPROF is profiling process 567, command: ./histogram
Elapsed time: 0.49
















*
*
*




*
**


*



*
*
**

**
*
*
***
****
****
*
*
******
****
**

****
****
**
**
******
*******
**********
*****
*********
********
***********
*****************
**********
************
*****************
********
***********
**************
***************
******************
****************
***************
**********************
*************
************************
****************************
****************************
*********************************
********************
********************************
*******************
**************************************
******************************
***********************************
*************************************
*************************************


Редукция

Дан массив А из N элементов (задаются случайно). Выполнить редукцию массива А с базовой операцией min.

In [3]:
%%writefile reduction.cu
#include <cstdlib>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <cstdio>
#include <iostream>
#include <ctime>

constexpr auto block_size = 256;

__device__ int min_g(const int a, const int b)
{
	if(a==0 || b==0)
	{
		return INT32_MAX;
	}

	if (a < b)
	{
		return a;
	}
	return b;
}

__global__ void reduce5(const int* in_data, int* out_data)
{
	__shared__ int data[block_size];
	const int tid = static_cast<int>(threadIdx.x);
	const int i = static_cast<int>(2 * blockIdx.x * blockDim.x + threadIdx.x);
	data[tid] = min_g(in_data[i], in_data[i + blockDim.x]);
	__syncthreads();
	for (int s = static_cast<int>(blockDim.x) / 2; s > 32; s >>= 1)
	{
		if (tid < s)
			data[tid] = min_g(data[tid], data[tid + s]);
		__syncthreads();
	}
	if (tid < 32)
	{
		data[tid] = min_g(data[tid], data[tid + 32]);
		data[tid] = min_g(data[tid], data[tid + 16]);
		data[tid] = min_g(data[tid], data[tid + 8]);
		data[tid] = min_g(data[tid], data[tid + 4]);
		data[tid] = min_g(data[tid], data[tid + 2]);
		data[tid] = min_g(data[tid], data[tid + 1]);
	}
	if (tid == 0)
		out_data[blockIdx.x] = data[0];
}

int reduce(const int* data, const int n)
{
	const auto matrix_size = n * sizeof(int);

	int* sums = nullptr;
	int* data_cuda = nullptr;

	const int num_blocks = n / block_size;
	int res = INT32_MAX;

	cudaMalloc(reinterpret_cast<void**>(&data_cuda), matrix_size);
	cudaMalloc(reinterpret_cast<void**>(&sums), matrix_size);
	cudaMemcpy(data_cuda, data, matrix_size, cudaMemcpyHostToDevice);

	reduce5 <<< dim3(num_blocks), dim3(block_size) >>> (data_cuda, sums);

	if (num_blocks > block_size)
	{
		res = reduce(sums, num_blocks);
	}
	else
	{
		const auto sums_host = new int[num_blocks];
		cudaMemcpy(sums_host, sums, num_blocks * sizeof(int), cudaMemcpyDeviceToHost);
		for (int i = 0; i < num_blocks; i++)
		{
			res = res < sums_host[i] ? res : sums_host[i];//проверяем оставшиеся части
		}
		delete[] sums_host;
	}
	cudaFree(sums);
	return res;
}

bool check_min(const int* a, const int actual_min, const int n)
{
	auto expected_min = a[0];
	for (auto i = 0; i < n; i++)
	{
		if (expected_min > a[i])
		{
			expected_min = a[i];
		}
	}

	std::cout << "Expected min: " << expected_min << std::endl;

	return expected_min == actual_min;
}

int main(int argc, char* argv[])
{
	srand(time(nullptr));
	constexpr auto matrix_len = 1024 * 1024;
	constexpr auto matrix_size = static_cast<int>(matrix_len) * sizeof(int);
	const auto a = static_cast<int*>(malloc(matrix_size));
	for (int i = 0; i < matrix_len; i++)
	{
		a[i] = rand() % matrix_len + block_size;
	}

	auto actual_min = reduce(a, matrix_len);
	std::cout << "Actual min_g = " << actual_min << ". Result is right: " << (check_min(a, actual_min, matrix_len) == 1 ? "true" : "false");

	free(a);

	return EXIT_SUCCESS;
}


Writing reduction.cu


In [4]:
!nvcc reduction.cu -o reduction -Wno-deprecated-gpu-targets
!nvprof ./reduction

==699== NVPROF is profiling process 699, command: ./reduction
Actual min_g = 260. Result is right: Expected min: 256
false==699== Profiling application: ./reduction
==699== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.61%  850.11us         1  850.11us  850.11us  850.11us  [CUDA memcpy HtoD]
                    7.64%  70.912us         2  35.456us  5.6000us  65.312us  reduce5(int const *, int*)
                    0.53%  4.8960us         1  4.8960us  4.8960us  4.8960us  [CUDA memcpy DtoD]
                    0.22%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   99.56%  436.35ms         4  109.09ms  3.8720us  436.05ms  cudaMalloc
                    0.25%  1.0989ms         3  366.30us  20.179us  1.0542ms  cudaMemcpy
                    0.10%  443.15us         1  443.15us  443.15us  443.15us  cuDeviceTotalMem
                    0.04%  164.48us       101  1.6280us     160ns